In [36]:
import requests
#資料抓取並成為字典
def download_data()->dict:
    url = 'https://opendata.cwb.gov.tw/fileapi/v1/opendataapi/F-C0032-001?Authorization=rdec-key-123-45678-011121314&format=JSON'

    respone = requests.get(url)
    if respone.status_code == 200:
        print("下載成功")
    return respone.json()



In [37]:
#資料解析成CSV
json_data = download_data()
def jsonDict_csvList(json)->list[dict]:
    '''
     - 傳入json的資料結構
     - 取出需要資料
     - 組合成list[dict]
    '''
    location = json['cwbopendata']['dataset']['location']
    weather_list = []
    for item in location:
        city_item = {}
        city_item['城市'] = item['locationName']
        city_item['啟始時間'] = item['weatherElement'][1]['time'][0]['startTime']
        city_item['結束時間'] = item['weatherElement'][1]['time'][0]['endTime']
        city_item['最高溫度'] = float(item['weatherElement'][1]['time'][0]['parameter']['parameterName'])
        city_item['最低溫度'] = float(item['weatherElement'][2]['time'][0]['parameter']['parameterName'])
        city_item['感覺'] = item['weatherElement'][3]['time'][0]['parameter']['parameterName']
        city_item['城市'] = item['locationName']
        weather_list.append(city_item)
    return weather_list

下載成功


In [38]:
import csv
def save_csv(data:list[dict],fileName)->None:
    """
     - 將lsit[dict]儲存為
     - 參數fileName要儲存的檔名
    """

    with open(fileName,mode='w', encoding='utf-8',newline='') as f:
        filenames = ['城市','啟始時間','結束時間','最高溫度','最低溫度','感覺']
        writer = csv.DictWriter(f,fieldnames=filenames)
        writer.writeheader()
        writer.writerows(data)

In [39]:
from datetime import datetime 
import pytz
def get_csvName()->str:
    '''
     - 取得台灣時間year-month-day
    '''
    taiwan_timezone = pytz.timezone('Asia/Taipei')
    current_time = datetime.now(taiwan_timezone)
    fileName = f'今日天氣 {current_time.year}-{current_time.month}-{current_time.day}  {current_time.hour}:{current_time.minute}:{current_time.second}.csv'
    return fileName

In [40]:
csvFileName = get_csvName()
csvFileName

'今日天氣 2023-8-12  14:29:15.csv'

In [41]:
import os
def check_file_exist()->bool:
    csvFileName = get_csvName()
    current_cwd = os.path.abspath(os.getcwd())
    abs_file_path = os.path.join(current_cwd,'data',csvFileName)
    if os.path.exists(abs_file_path):
        return True
    else:
        return False
check_file_exist()

False

In [42]:
#主程式
if not check_file_exist():
    print('不存在')
    json_data = download_data()#下載資料
    csv_list = jsonDict_csvList(json_data)#處理資料
    is_save = save_csv(csv_list,csvFileName)#儲存資料
    if is_save:
        print('下載成功')


不存在
下載成功
